In [ ]:
# Third-party
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

In [ ]:
# orig_g = Table.read('../data/R1500_z_750_cyl_rv.fits', memmap=True)
# mask = np.isfinite(orig_g['radial_velocity']) & ((orig_g['parallax'] / orig_g['parallax_error']) > 6)
# mask.sum()

g = GaiaData('../data/R1500_z_750_cyl_rv.fits')

In [ ]:
c = g.skycoord

In [ ]:
np.isfinite(g.e_bp_min_rp_val).sum(), len(g)

In [ ]:
gal = c.galactic[(g.visibility_periods_used >= 8)]
gal.set_representation_cls('cartesian')

fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
axes[0,0].plot(gal.u, gal.v, marker=',', alpha=0.1, linestyle='none')
axes[1,0].plot(gal.u, gal.w, marker=',', alpha=0.1, linestyle='none')
axes[1,1].plot(gal.v, gal.w, marker=',', alpha=0.1, linestyle='none')
axes[0,1].set_visible(False)

axes[0,0].set_xlim(-500, 500)
axes[0,0].set_ylim(-500, 500)

fig.tight_layout()

In [ ]:
plt.hist(g.phot_g_mean_mag[np.isfinite(g.phot_g_mean_mag)], 
         bins=np.linspace(8, 14., 32));
plt.axvline(12.6)

In [ ]:
m = 12.6*u.mag
M = 5.*u.mag
coord.Distance(distmod=m-M)

Note: to be somewhat complete with RVs, we should really only go down to abs. mag ~ 5

Assign reddening by nearest neighbor:

In [ ]:
# idx, sep2d, dist3d = c.match_to_catalog_3d(c, nthneighbor=2)

no_ext = np.isnan(g.e_bp_min_rp_val) | np.isnan(g.a_g_val)
idx, sep2d, dist3d = c[no_ext].match_to_catalog_sky(c, nthneighbor=2)

In [ ]:
e_bprp = g.e_bp_min_rp_val.value
a_g = g.a_g_val.value

e_bprp[no_ext] = e_bprp[idx]
a_g[no_ext] = a_g[idx]

In [ ]:
bprp = (g.phot_bp_mean_mag - g.phot_rp_mean_mag)
G = g.phot_g_mean_mag

bprp0 = bprp - e_bprp*u.mag
G0 = G - a_g*u.mag

In [ ]:
# F5–K5
tmp_fgk_mask = (g.teff_val.value < 6500.) & (g.teff_val.value > 4400)
test_mask = tmp_fgk_mask & (g.distance < 200*u.pc)

In [ ]:
bins = (np.arange(-0.5, 2.5+1e-5, 0.02),
        np.arange(-1, 12+1e-5, 0.02))

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

ax = axes[0]

# H, xe, ye = np.histogram2d(bprp[test_mask], (G - g.distmod)[test_mask], 
H, xe, ye = np.histogram2d(bprp0[test_mask], (G0 - g.distmod)[test_mask], 
                           bins=bins)
ax.pcolormesh(xe, ye, H.T, cmap='Greys',
              norm=mpl.colors.LogNorm(vmin=1E0, vmax=1E4))

nodes = np.array([[-0.05, 1.1],
                  [-0.05, 1.65],
                  [0.55, 3.65],
                  [0.77, 5],
                  [1, 6],
                  [1.2, 6],
                  [1, 5.2],
                  [0.65, 3.3],
                  [0.4, 2.5],
                  [-0.05, 1.1]])
ax.plot(nodes[:, 0], nodes[:, 1], marker='.')
fgk_path = mpl.patches.Path(nodes)
ax.axhline(6, zorder=-100)

ax = axes[1]

# H, xe, ye = np.histogram2d(bprp, G - g.distmod, 
H, xe, ye = np.histogram2d(bprp0, G0 - g.distmod, 
                           bins=bins)
ax.pcolormesh(xe, ye, H.T, cmap='Greys',
              norm=mpl.colors.LogNorm(vmin=1E0, vmax=1E4))
ax.plot(nodes[:, 0], nodes[:, 1], marker='.')

ax.set_ylim(12, -1)
ax.set_xlim(-0.5, 2.5)

fig.tight_layout()

In [ ]:
xy = np.vstack((bprp0, G0 - g.distmod)).T.value

In [ ]:
fgk_mask = fgk_path.contains_points(xy)
fgk_mask.sum()

In [ ]:
gal_fgk = c.galactic[fgk_mask]
gal_fgk.set_representation_cls('cartesian')

fig, axes = plt.subplots(2, 2, figsize=(8, 7.2), sharex=True, sharey=True)
axes[0,0].plot(gal_fgk.u, gal_fgk.v, 
               marker=',', alpha=0.2, linestyle='none')
axes[1,0].plot(gal_fgk.u, gal_fgk.w, 
               marker=',', alpha=0.2, linestyle='none')
axes[1,1].plot(gal_fgk.v, gal_fgk.w, 
               marker=',', alpha=0.2, linestyle='none')
axes[0,1].set_visible(False)

axes[0,0].set_xlim(-500, 500)
axes[0,0].set_ylim(-500, 500)

fig.tight_layout()

---

## Hexagons

In [ ]:
# internal triangle side length
h = 120. # pc

In [ ]:
def get_hexagons(h):
    a = np.sqrt(3)/2 * h # inner side
    
    pas = dict() # keyed by "ring"
    pas[0] = list()
    pas[1] = list()
    pas[2] = list()
#     pas[3] = list()
    
    pa0 = mpl.patches.RegularPolygon((0., 0.), numVertices=6, 
                                     radius=h, orientation=np.pi/2)
    pas[0].append(pa0.get_verts())

    for ang in np.arange(0, 360, 60)*u.deg:
        # Ring 1
        xy0 = 2*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[1].append(pa.get_verts())
        
        # Ring 2
        xy0 = 4*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
        xy0 = 3*h * np.array([np.cos(ang+120*u.deg), np.sin(ang+120*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
#         # Ring 3
#         xy0 = 6*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
#         pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
#                                         radius=h, orientation=np.pi/2)
#         pas[3].append(pa.get_verts())
        
#         xy0 = 5*a/np.cos(20*u.deg) * np.array([np.cos(ang+110*u.deg), np.sin(ang+110*u.deg)])
#         pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
#                                         radius=h, orientation=np.pi/2)
#         pas[3].append(pa.get_verts())
        
    return pas

In [ ]:
hexs = get_hexagons(h)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for k in hexs.keys():
    for pa in hexs[k]:
        pa = mpl.patches.Polygon(pa, facecolor='none', edgecolor='#333333')
        ax.add_patch(pa)

        
ax.plot(gal_fgk.u, gal_fgk.v, 
        marker=',', alpha=0.1, color='k',
        linestyle='none', zorder=100)
        
lim = 650
ax.set_xlim(-lim, lim)
ax.set_ylim(-lim, lim)

ax.set_xlabel('$x$ [pc]')
ax.set_ylabel('$y$ [pc]')

---

In [ ]:
fgk_cyl = gal_fgk.transform_to(coord.Galactocentric(galcen_distance=8*u.kpc,
                                                    z_sun=0*u.kpc))
fgk_cyl.set_representation_cls('cylindrical')

xyz = np.vstack((gal_fgk.u.to(u.pc).value, 
                 gal_fgk.v.to(u.pc).value,
                 gal_fgk.w.to(u.pc).value)).T

# UVW = np.vstack((gal_fgk.U.to(u.km/u.s).value, 
#                  gal_fgk.V.to(u.km/u.s).value, 
#                  gal_fgk.W.to(u.km/u.s).value)).T

UVW = np.vstack((fgk_cyl.d_rho.to(u.km/u.s).value, 
                 - ((fgk_cyl.rho * fgk_cyl.d_phi).to(u.km/u.s, u.dimensionless_angles()).value + 220.), 
                 fgk_cyl.d_z.to(u.km/u.s).value)).T

In [ ]:
for k in hexs.keys():
    for b, pa in enumerate(hexs[k]):
        hex_mask = mpl.patches.Path(pa).contains_points(xyz[:, :2])
        print(hex_mask.sum())

        lim = 150
        bins = np.linspace(-lim, lim, 128)

        fig, axes = plt.subplots(2, 2, figsize=(8, 7.2))
        
        for a, (i, j) in zip([0, 2, 3], [(0, 1), (0, 2), (1, 2)]):
            ax = axes.flat[a]
            H, xe, ye = np.histogram2d(UVW[hex_mask,i], UVW[hex_mask,j], bins=bins)
            ax.pcolormesh(xe, ye, H.T, 
                          norm=mpl.colors.LogNorm(), 
                          cmap='magma', vmin=1, vmax=3E2)
            ax.set_xlim(-lim, lim)
            ax.set_ylim(-lim, lim)
        
        axes[0, 0].set_ylabel('$-v_\phi-220$')
        axes[1, 0].set_ylabel('$v_z$')
        axes[1, 0].set_xlabel('$v_R$')
        axes[1, 1].set_xlabel('$-v_\phi-220$')
        axes[0, 0].xaxis.set_ticklabels([])
        axes[1, 1].yaxis.set_ticklabels([])
        
        # axes[0,1].set_visible(False)
        ax = axes[0,1]
        for k_ in hexs.keys():
            for pa_ in hexs[k_]:
                pa_ = mpl.patches.Polygon(pa_, facecolor='none', edgecolor='#333333')
                ax.add_patch(pa_)
                
        ax.add_patch(mpl.patches.Polygon(pa, facecolor='#333333', edgecolor='#333333'))
        ax.set_xlim(-500, 500)
        ax.set_ylim(-500, 500)
        ax.set_xlabel('$x$ [pc]')
        ax.set_ylabel('$y$ [pc]')

        fig.tight_layout()
        fig.savefig('hexagon-plots/{0}-{1:02d}.png'.format(k, b), dpi=250)
        plt.close(fig)


```
convert -delay 30 -loop 0 1-*.png ring1.gif
convert -delay 30 -loop 0 2-*.png ring2.gif
convert -delay 30 -loop 1 *.png all.gif
```

In [ ]:
line_cmd = 'convert -delay 30 -loop 0 {0} full-line{1}.gif'
for k, a in enumerate([[(2,0),(1,0),(0,0),(1,3),(2,6)],
                       [(2,2),(1,1),(0,0),(1,4),(2,8)],
                       [(2,4),(1,2),(0,0),(1,5),(2,10)]]):
    print(line_cmd.format(' '.join(['{0}-{1:02d}.png'.format(i, j) for i, j in a]), k))